In [2]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 15.7 MB/s eta 0:00:00


In [3]:
import os
from sentence_transformers import SentenceTransformer
from transformers import pipeline
import faiss
import numpy as np

In [4]:
from google.colab import drive
drive.mount('/content/drive')
folder_path = "/content/drive/MyDrive/RAG/Data/rhymes"

Mounted at /content/drive


In [5]:
# Load rhyme text files
def load_rhyme_texts(path="./rhymes"):
    texts = []
    for filename in os.listdir(path):
        if filename.endswith(".txt"):
            with open(os.path.join(path, filename), "r", encoding="utf-8") as f:
                texts.append(f.read())
    return texts

docs = load_rhyme_texts(folder_path)

In [7]:
# Embed using SentenceTransformer
embedder = SentenceTransformer('all-MiniLM-L6-v2')
doc_embeddings = embedder.encode(docs).astype('float32')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
# Index with FAISS
index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(doc_embeddings)

In [9]:
# Basic QA model (you can switch to RAG if you want)
qa = pipeline("text2text-generation", model="google/flan-t5-base")

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


(…)b68e004ad934361fb35b9b2bd50b45ea90790fc8:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cpu


In [18]:
# Query
def query_rhymes(prompt, top_k=3):
    query_emb = embedder.encode([prompt]).astype('float32')
    _, indices = index.search(query_emb, top_k)
    context = "\n".join([docs[i] for i in indices[0]])
    full_prompt = f"Given the following nursery rhymes:\n{context}\n\nQuestion: {prompt}"
    response = qa(full_prompt, max_length=500, do_sample=False)[0]['generated_text']
    return response

if __name__ == "__main__":
    while True:
        query = input("Ask about a rhyme (or press Enter to exit): ")
        if not query.strip():
            break
        result = query_rhymes(query)
        print("Answer:", result)

Ask about a rhyme (or press Enter to exit): whom did ba ba black sheep give wool to?
Answer: the master
Ask about a rhyme (or press Enter to exit): what is my name
Answer: john jacob jingleheimer schmidt
Ask about a rhyme (or press Enter to exit): what is the time
Answer: six o’clock
Ask about a rhyme (or press Enter to exit): what is cos 90
Answer: a tadpole
Ask about a rhyme (or press Enter to exit): what is roshni
Answer: a slut
Ask about a rhyme (or press Enter to exit): You are ill mannered 
Answer: You are a good maid.
Ask about a rhyme (or press Enter to exit): Your answers are not reliable and absurd
Answer: Number one took the cookie from the cookie jar! Who me? Yes, you! Couldn’t be! Then who took the cookie from the cookie jar! Number two took the cookie from the cookie jar! Who me? Yes, you! Couldn’t be! Then who took the cookie from the cookie jar! Number three took the cookie from the cookie jar! Who me? Yes, you! Couldn’t be! Then who took the cookie from the cookie jar!